# Ayudantía 2

## Programación Funcional

Vicente Águila y Paul Heinsohn

**Form:** https://docs.google.com/forms/d/1rublnCunwYWYe2QxARiND1hS9WJ9jKLE6WJilA7rRvE

## ¿Por qué?

* Existen lenguajes de programación que utilizan este paradigma (Erlang)
* Procesamiento de información optimizada

________________________________________________________________________________

## Introducción
<br>
<div style="text-align: justify">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Llevas todo el día estudiando la materia de Programación funcional que ya no das más, por lo que decides ver <strike>muchas</strike> una película, sin embargo, hay tantas películas buenas como ’Buscando a Enzo’, ’Las 50 Sombras de Hernan’, ’Lo Que Benja Se Llevó', etc. que no logras decidirte. Luego de un rato, llegas a la brillante idea de crear un programa que te ayude a resolverlo, para esto ingresas a Prograflix a ver como puedes comenzar.

<img src="img/prograflix.png">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;El objetivo es realizar un sistema de consultas que te permita obtener información sobre películas y actores. Para esto, obtienes mágicamente la base de datos de <i>Prograflix</i> organizada en dos archivos especificados a continuación:
    
<ul>
    <li><b>peliculas.txt:</b> Archivo de texto que contiene la información de las películas que puedes ver, de la forma: id, nombre, puntuacion, fecha_estreno, genero</li><br>
    <li><b>actores.txt:</b> Archivo de texto que alberga la información de los artistas que trabajaron en las películas. Está distribuido de la forma: nombre_pelicula, nombre_actor, nombre_personaje.</li>
</ul>

Con esta información debes ser capaz de resolver las consultas especificadas siempre y cuando se cumpla lo siguiente:

<ul>
    <li>Tu solución debe ser realizada con <b>programación funcional</b></li><br>
    <li>Quieres que tu programa no almacene el archivo en memoria, ergo, la lectura tendrá que ser sólamente mediante el uso de generadores</li><br>
    <li>Cada película debe tener su propio id que debe estar implementado con <b>generadores</b>. El id es distinto al que trae la base de datos, para que puedas distinguirla con tu programa.</li><br>
</ul>

Luego, lo primero que debemos hacer es crear el id para las películas:
</div>

In [ ]:
def id_pelicula():  # Debido al punto N°3
    id_ = 0
    while True:
        yield id_
        id_ += 1

Procedemos a definir las películas y los actores:

In [ ]:
from collections import namedtuple


Pelicula = namedtuple("Pelicula", ["id", "nombre", "puntuacion", "fecha_estreno", "genero"])
Actor = namedtuple("Actor", "nombre_pelicula nombre_actor nombre_personaje")

<div style="text-align: justify">Ahora faltaría abrir los archivos para instanciar los actores y las películas. Hasta ahora solo hemos trabajado los archivos guardándolos en memoria mediante "for's" tediosos... pero qué tal si lo hacemos con listas por compresión:
</div>

In [ ]:
limpiador = lambda x: x.strip().split(",")
gen_id = id_pelicula()


with open("archivos/peliculas.txt", "r", encoding="utf-8") as linea:
    PELICULAS = [Pelicula(next(gen_id), *limpiador(p)[1:]) for p in linea]
        
with open("archivos/actores.txt", "r", encoding="utf-8") as linea:
    ACTORES = [Actor(*limpiador(a)) for a in linea]

In [ ]:
for i, elem in enumerate(PELICULAS, 1):
    if i > 5:
        break
    print("{}: {}\n".format(i, elem))


In [ ]:
for i, elem in enumerate(ACTORES):
    if i > 4:
        break
    print("{}: {}\n".format(i + 1, elem))

________________________________________________________________________________
Dentro de las consultas que nuestro programa debe soportar veremos los siguientes:

* Crear el método <b>popular</b> que dado un número <b>n</b>, retorne todas las películas que tienen un rating superior a dicho valor.

In [ ]:
def popular(pelis, n):
    return filter(lambda x: float(x.puntuacion) >= n, pelis)

In [ ]:
print([top.nombre for top in popular(PELICULAS, 20)])
print('-' * 35)
print('cosas', 'separadas', 'por', 'forma', 'normal', 'se', 'ven', 'asi')
print('-' * 35)
print('cosas', 'separadas', 'por', 'coma', 'se', 'ven', 'asi', sep=",")
print('-' * 35)
print(*[top.nombre for top in popular(PELICULAS, 20)])
print('-' * 35)
# Y aquí explicar lo del sep, porque los chicos pueden no saber la magia del sep
print('cosas', 'separadas', 'por', 'guionBajo', 'se', 'ven', 'asi', sep='_')
print('-' * 35)
print(*[top.nombre for top in popular(PELICULAS, 20)], sep='\n')

________________________________________________________________________________
* Cree la función **actores_genero** que retorne el **nombre** de todos los actores que actuaron en una película del **género** especificado. 

In [ ]:
def actores_genero(genero, actores, peliculas):
    peliculas_genero = filter(lambda p: p.genero == genero, peliculas)
    nombres_p_genero = list(map(lambda p_gen: p_gen.nombre, peliculas_genero))
    
    actores_peliculas = filter(lambda a: a.nombre_pelicula in nombres_p_genero, actores)
    actores_genero = map(lambda a_gen: a_gen.nombre_actor, actores_peliculas)
    return actores_genero

# Es lo mismo que hacer lo siguiente

def actores_genero_simplificado(genero, actores, peliculas):
    peliculas_genero = list(map(lambda p_gen: p_gen.nombre, filter(lambda p: p.genero == genero, peliculas)))
    return map(lambda a_gen: a_gen.nombre_actor, filter(lambda a: a.nombre_pelicula in peliculas_genero, actores))

In [ ]:
print(*list(actores_genero("Comedy", ACTORES, PELICULAS))[:9], sep="\n")
print("-" * 35)
print(*actores_genero("Comedy", ACTORES, PELICULAS), sep="\n")

________________________________________________________________________________
* Crear el método <b>puntuacion_actor</b> que dado el <b>nombre de 'Hugh Jackman'</b>, retorna el promedio de la puntuación de las películas en las que ha participado.

In [ ]:
from functools import reduce


def puntuacion_actor(peliculas, actores, n_actor):
    nombre_peliculas_actor = [a.nombre_pelicula for a in filter(lambda actor: actor.nombre_actor == n_actor, actores)]
    
    peliculas_actor = filter(lambda pelicula: pelicula.nombre in nombre_peliculas_actor, peliculas)
    return reduce(lambda x, y: x + y, map(lambda p: float(p.puntuacion), peliculas_actor)) / len(nombre_peliculas_actor)
    

In [ ]:
print("Hugh Jackman's rating: {}".format(puntuacion_actor(PELICULAS, ACTORES, "Hugh Jackman")))

________________________________________________________________________________________________________________________________
* Crear el método **puntuaciones_actores** que retorne el promedio de la puntuación de las películas en que cada actor ejerce su trabajo.

In [ ]:
def puntuaciones_actores(peliculas, actores):
    nombre_actores = {a.nombre_actor for a in actores}
    return {n: puntuacion_actor(peliculas, actores, n) for n in nombre_actores}

In [ ]:
puntuaciones_actores(PELICULAS, ACTORES)

______________________________________________________________________________________________________________________________
En caso de que queramos crear una función generadora que no almacene la menor cantidad de memoria podría ser útil la siguiente función que deberemos llamar cada vez que queramos hacer una consulta:

In [ ]:
def abrir_archivos(ruta_archivo):
    with open(ruta_archivo, "r", encoding="utf-8") as file:
        for i in file:
            yield limpiador(i)

In [ ]:
abridor = abrir_archivos("archivos/peliculas.txt")
print(next(abridor))
print(next(abridor))
print(next(abridor))

In [ ]:
for linea in abrir_archivos("archivos/peliculas.txt"):
    print(linea)